In [12]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import decoupler as dc
import plotnine as p9
import liana as li
import numpy as np

from pathlib import Path
from scipy.sparse import issparse, csr_matrix
from liana.method import cellphonedb, natmi
from mousipy import translate
import datetime

# loading data

# Merfish

In [35]:
# nemelo by se najit hodne interakci
adata = sc.read_h5ad("../PRIMUS/data/33_data/transfer/Sofie_bc_prace/data_true/merscope_final_annotated_subseted_gene_symbol.h5ad")
adata

adata.obs.columns
adata.var
adata.layers['raw'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.layers['lognorm'] = adata.X.copy()
#just to look how it looks like, especially if we have Ensemble IDs or gene names
adata.to_df(layer="raw")

adata.var.index = adata.var["gene_name"]

#control that the names are now symbols. If not, try adata.var_names = adata.var["gene_name"]

adata.to_df(layer="raw")
print(adata.var_names[:5])

print(li.mt.show_methods())
print(li.resource.show_resources())

Index(['Htr7', 'Gzmk', 'Arhgap36', 'Sema3c', 'Rxrg'], dtype='object', name='gene_name')
         Method Name Magnitude Score Specificity Score  \
0        CellPhoneDB        lr_means   cellphone_pvals   
0         Connectome       expr_prod     scaled_weight   
0             log2FC            None          lr_logfc   
0              NATMI       expr_prod       spec_weight   
0  SingleCellSignalR         lrscore              None   
0           CellChat        lr_probs    cellchat_pvals   
0     Rank_Aggregate  magnitude_rank  specificity_rank   
0     Geometric Mean       lr_gmeans       gmean_pvals   

                                           Reference  
0  Efremova, M., Vento-Tormo, M., Teichmann, S.A....  
0  Raredon, M.S.B., Yang, J., Garritano, J., Wang...  
0  Dimitrov, D., Türei, D., Garrido-Rodriguez, M....  
0  Hou, R., Denisenko, E., Ong, H.T., Ramilowski,...  
0  Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ...  
0  Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha...  
0  D

In [36]:
adata.X = adata.layers['raw']
humanized_adata = translate(adata,stay_sparse=True)
humanized_adata.var

humanized_adata.layers['raw'] = humanized_adata.X.copy()
sc.pp.normalize_total(humanized_adata, target_sum=1e4)
sc.pp.log1p(humanized_adata)
humanized_adata.layers['lognorm'] = humanized_adata.X.copy()

meta = pd.DataFrame({
    'Cell': humanized_adata.obs_names,
    'cell_type': humanized_adata.obs['cell_type']
})
meta.to_csv("../output/cpdb/merfish_meta.txt", sep='\t', index=False)

#something like this will be important for visiumHD because the csv is gonna be HUGE
#not tested
adata_to_use = humanized_adata.copy()
adata_to_use.X = adata_to_use.layers['raw']  # or use .X if raw is not available
adata_to_use.var_names_make_unique()

# Create the DataFrame
counts = pd.DataFrame(
    adata_to_use.X.T.toarray() if hasattr(adata_to_use.X, "toarray") else adata_to_use.X.T,
    index=adata_to_use.var_names,
    columns=adata_to_use.obs_names
)

# Save as HDF5
counts.to_hdf("../output/cpdb/merfish_counts.h5", key="counts", mode="w")

100%|██████████| 1122/1122 [00:02<00:00, 459.55it/s]


Found direct orthologs for 1056 genes.
Found multiple orthologs for 3 genes.
Found no orthologs for 55 genes.
Found no index in biomart for 8 genes.


  0%|          | 0/1 [00:00<?, ?it/s]/usr/lib/python3/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
                                             

In [18]:
adata.obs

,organism_ontology_term_id,donor_id,development_stage_ontology_term_id,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,...,is_primary_data,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,observation_joinid
1062701771071696533487975728882770248,NCBITaxon:10090,C57BL6J-2,MmusDv:0000155,PATO:0000384,na,PATO:0000461,UBERON:0000955,CL:4023051,EFO:0008992,na,...,True,vascular leptomeningeal cell,MERFISH,normal,Mus musculus,male,brain,na,9-week-old stage,G;aLxtIU+O
140347902001698538054266201822142859661,NCBITaxon:10090,C57BL6J-2,MmusDv:0000155,PATO:0000384,na,PATO:0000461,UBERON:0000955,CL:4023051,EFO:0008992,na,...,True,vascular leptomeningeal cell,MERFISH,normal,Mus musculus,male,brain,na,9-week-old stage,&5HvbDNa2`
17912510556127714390369784436317555462,NCBITaxon:10090,C57BL6J-2,MmusDv:0000155,PATO:0000384,na,PATO:0000461,UBERON:0000955,CL:0000127,EFO:0008992,na,...,True,astrocyte,MERFISH,normal,Mus musculus,male,brain,na,9-week-old stage,Vhh72ga0fu
202887872709858303957792369114911560476,NCBITaxon:10090,C57BL6J-2,MmusDv:0000155,PATO:0000384,na,PATO:0000461,UBERON:0001950,CL:0002453,EFO:0008992,na,...,True,oligodendrocyte precursor cell,MERFISH,normal,Mus musculus,male,neocortex,na,9-week-old stage,9zdojI7}a9
218006636060783985610825029641487829718,NCBITaxon:10090,C57BL6J-2,MmusDv:0000155,PATO:0000384,na,PATO:0000461,UBERON:0000955,CL:0000129,EFO:0008992,na,...,True,microglial cell,MERFISH,normal,Mus musculus,male,brain,na,9-week-old stage,b|i|;7j9Us
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5197191772708129939279235494138010912,NCBITaxon:10090,C57BL6J-2,MmusDv:0000155,PATO:0000384,na,PATO:0000461,UBERON:0000955,CL:0000617,EFO:0008992,na,...,True,GABAergic neuron,MERFISH,normal,Mus musculus,male,brain,na,9-week-old stage,B%R+ZiqX=?
57045319109046809920494497966207932917,NCBITaxon:10090,C57BL6J-2,MmusDv:0000155,PATO:0000384,na,PATO:0000461,UBERON:0000955,CL:0002453,EFO:0008992,na,...,True,oligodendrocyte precursor cell,MERFISH,normal,Mus musculus,male,brain,na,9-week-old stage,DBXdg6XE8v
77789447341428476484230796188642956517,NCBITaxon:10090,C57BL6J-2,MmusDv:0000155,PATO:0000384,na,PATO:0000461,UBERON:0000955,CL:0000617,EFO:0008992,na,...,True,GABAergic neuron,MERFISH,normal,Mus musculus,male,brain,na,9-week-old stage,vs!Aa9qPH!
95287028017647642274874978777769935669,NCBITaxon:10090,C57BL6J-2,MmusDv:0000155,PATO:0000384,na,PATO:0000461,UBERON:0000955,CL:0000115,EFO:0008992,na,...,True,endothelial cell,MERFISH,normal,Mus musculus,male,brain,na,9-week-old stage,F@>Bf{^d$=


In [37]:
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method

#all filtered - nenasel, nema tf


print("cellphone START")
print(datetime.datetime.now())
cpdb_results = cpdb_statistical_analysis_method.call(
    cpdb_file_path="../data/cellphonedb.zip",  # use default database (or provide path to custom .zip)
    meta_file_path="../output/cpdb/merfish_meta.txt",
    counts_file_path="../output/cpdb/merfish_counts.h5",
    counts_data="gene_name",  # or 'ensembl'
    output_path="../output/cpdb/merfish_cpdb")
print("cellphone END")
print(datetime.datetime.now())

cellphone START
2025-04-28 22:08:02.517574
Reading user files...
The following user files were loaded successfully:
../output/cpdb/merfish_counts.h5
../output/cpdb/merfish_meta.txt
[ ][CORE][28/04/25-22:08:03][INFO] [Cluster Statistical Analysis] Threshold:0.1 Iterations:1000 Debug-seed:-1 Threads:4 Precision:3
[ ][CORE][28/04/25-22:08:03][INFO] Running Real Analysis
[ ][CORE][28/04/25-22:08:03][INFO] Running Statistical Analysis


100%|██████████| 1000/1000 [03:13<00:00,  5.18it/s]


[ ][CORE][28/04/25-22:11:19][INFO] Building Pvalues result
[ ][CORE][28/04/25-22:11:19][INFO] Building results
Saved deconvoluted to ../output/cpdb/merfish_cpdb/statistical_analysis_deconvoluted_04_28_2025_221119.txt
Saved deconvoluted_percents to ../output/cpdb/merfish_cpdb/statistical_analysis_deconvoluted_percents_04_28_2025_221119.txt
Saved means to ../output/cpdb/merfish_cpdb/statistical_analysis_means_04_28_2025_221119.txt
Saved pvalues to ../output/cpdb/merfish_cpdb/statistical_analysis_pvalues_04_28_2025_221119.txt
Saved significant_means to ../output/cpdb/merfish_cpdb/statistical_analysis_significant_means_04_28_2025_221119.txt
cellphone END
2025-04-28 22:11:19.927620


# Xenium

In [38]:
##loading data
adata = sc.read_h5ad("../PRIMUS/data/33_data/transfer/Sofie_bc_prace/data_true/adata_xenium_final_annotated.h5ad")
adata

AnnData object with n_obs × n_vars = 2299 × 5006
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'region', 'z_level', 'cell_labels', 'n_genes', 'n_counts', 'cell_groups'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'log1p', 'spatialdata_attrs'
    obsm: 'spatial'
    layers: 'lognorm', 'raw'

In [ ]:
humanized_adata.obs

In [39]:
adata.X = adata.layers['raw']
humanized_adata = translate(adata,stay_sparse=True)
humanized_adata.var

humanized_adata.layers['raw'] = humanized_adata.X.copy()
sc.pp.normalize_total(humanized_adata, target_sum=1e4)
sc.pp.log1p(humanized_adata)
humanized_adata.layers['lognorm'] = humanized_adata.X.copy()

meta = pd.DataFrame({
    'Cell': humanized_adata.obs_names,
    'cell_type': humanized_adata.obs['cell_groups']
})
meta.to_csv("../output/cpdb/xenium_meta.txt", sep='\t', index=False)

#something like this will be important for visiumHD because the csv is gonna be HUGE
#not tested
adata_to_use = humanized_adata.copy()
adata_to_use.X = adata_to_use.layers['raw']  # or use .X if raw is not available
adata_to_use.var_names_make_unique()

# Create the DataFrame
counts = pd.DataFrame(
    adata_to_use.X.T.toarray() if hasattr(adata_to_use.X, "toarray") else adata_to_use.X.T,
    index=adata_to_use.var_names,
    columns=adata_to_use.obs_names
)

# Save as HDF5
counts.to_hdf("../output/cpdb/xenium_counts.h5", key="counts", mode="w")

100%|██████████| 5006/5006 [00:10<00:00, 459.91it/s]


Found direct orthologs for 4746 genes.
Found multiple orthologs for 76 genes.
Found no orthologs for 126 genes.
Found no index in biomart for 58 genes.


  0%|          | 0/18 [00:00<?, ?it/s]/usr/lib/python3/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [40]:
##nenaslo, prisna metoda

from cellphonedb.src.core.methods import cpdb_statistical_analysis_method
from cellphonedb.src.core.methods import cpdb_analysis_method


print("cellphone START")
print(datetime.datetime.now())
cpdb_results = cpdb_statistical_analysis_method.call(
    cpdb_file_path="../data/cellphonedb.zip",  # use default database (or provide path to custom .zip)
    meta_file_path="../output/cpdb/xenium_meta.txt",
    counts_file_path="../output/cpdb/xenium_counts.h5",
    counts_data="hgnc_symbol",  # or 'ensembl' or gene_name
    output_path="../output/cpdb/xenium_cpdb", threshold=0)

print("cellphone END")
print(datetime.datetime.now())

cellphone START
2025-04-28 22:12:20.964999
Reading user files...
The following user files were loaded successfully:
../output/cpdb/xenium_counts.h5
../output/cpdb/xenium_meta.txt
[ ][CORE][28/04/25-22:12:21][INFO] [Cluster Statistical Analysis] Threshold:0 Iterations:1000 Debug-seed:-1 Threads:4 Precision:3
[ ][CORE][28/04/25-22:12:21][INFO] Running Real Analysis
[ ][CORE][28/04/25-22:12:21][INFO] Running Statistical Analysis


100%|██████████| 1000/1000 [00:16<00:00, 60.94it/s]


[ ][CORE][28/04/25-22:12:40][INFO] Building Pvalues result
[ ][CORE][28/04/25-22:12:40][INFO] Building results
Saved deconvoluted to ../output/cpdb/xenium_cpdb/statistical_analysis_deconvoluted_04_28_2025_221240.txt
Saved deconvoluted_percents to ../output/cpdb/xenium_cpdb/statistical_analysis_deconvoluted_percents_04_28_2025_221240.txt
Saved means to ../output/cpdb/xenium_cpdb/statistical_analysis_means_04_28_2025_221240.txt
Saved pvalues to ../output/cpdb/xenium_cpdb/statistical_analysis_pvalues_04_28_2025_221240.txt
Saved significant_means to ../output/cpdb/xenium_cpdb/statistical_analysis_significant_means_04_28_2025_221240.txt
cellphone END
2025-04-28 22:12:41.743750


# Cosmx

In [3]:
adata = sc.read_h5ad("../PRIMUS/data/33_data/transfer/Sofie_bc_prace/data_true/cosmx_final_annotated.h5ad")
adata
adata.obs.cell_groups

1_1         Endo
2_1         VLMC
3_1        Astro
4_1         Endo
5_1         Endo
           ...  
311_130    Astro
312_130    Meis2
313_130     VLMC
314_130     VLMC
315_130     VLMC
Name: cell_groups, Length: 48556, dtype: category
Categories (23, object): ['Astro', 'CR', 'Endo', 'L2/3 IT', ..., 'Sncg', 'Sst', 'VLMC', 'Vip']

In [4]:
adata.X = adata.layers['counts']
humanized_adata = translate(adata,stay_sparse=True)
humanized_adata.var

humanized_adata.layers['counts'] = humanized_adata.X.copy()
sc.pp.normalize_total(humanized_adata, target_sum=1e4)
sc.pp.log1p(humanized_adata)
humanized_adata.layers['lognorm'] = humanized_adata.X.copy()

meta = pd.DataFrame({
    'Cell': humanized_adata.obs_names,
    'cell_type': humanized_adata.obs['cell_groups']
})
meta.to_csv("../output/cpdb/cosmx_meta.txt", sep='\t', index=False)

#something like this will be important for visiumHD because the csv is gonna be HUGE
#not tested
adata_to_use = humanized_adata.copy() ##zmena
adata_to_use.X = adata_to_use.layers['lognorm']  # or use .X if raw is not available
adata_to_use.var_names_make_unique()

# Create the DataFrame
counts = pd.DataFrame(
    adata_to_use.X.T.toarray() if hasattr(adata_to_use.X, "toarray") else adata_to_use.X.T,
    index=adata_to_use.var_names,
    columns=adata_to_use.obs_names
)

# Save as HDF5
counts.to_hdf("../output/cpdb/cosmx_counts.h5", key="counts", mode="w")

100%|██████████| 960/960 [00:02<00:00, 456.59it/s]


Found direct orthologs for 910 genes.
Found multiple orthologs for 10 genes.
Found no orthologs for 22 genes.
Found no index in biomart for 18 genes.


  0%|          | 0/10 [00:00<?, ?it/s]/home/blizkovs/.local/lib/python3.12/site-packages/mousipy/mousipy.py:205: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/home/blizkovs/.local/lib/python3.12/site-packages/mousipy/mousipy.py:205: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
 10%|█         | 1/10 [00:03<00:30,  3.43s/it]/home/blizkovs/.local/lib/python3.12/site-packages/mousipy/mousipy.py:205: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a

In [5]:
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method
from cellphonedb.src.core.methods import cpdb_analysis_method


print("cellphone START")
print(datetime.datetime.now())
cpdb_results = cpdb_statistical_analysis_method.call( 
    cpdb_file_path="../data/cellphonedb.zip",  # use default database (or provide path to custom .zip)
    meta_file_path="../output/cpdb/cosmx_meta.txt",
    counts_file_path="../output/cpdb/cosmx_counts.h5",
    counts_data="hgnc_symbol",  # or 'ensembl' or gene_name
    output_path="../output/cpdb/cosmx_cpdb", threshold=0)

print("cellphone END")
print(datetime.datetime.now())

cellphone START
2025-04-28 22:52:10.407955
Reading user files...
The following user files were loaded successfully:
../output/cpdb/cosmx_counts.h5
../output/cpdb/cosmx_meta.txt
[ ][CORE][28/04/25-22:52:11][INFO] [Cluster Statistical Analysis] Threshold:0 Iterations:1000 Debug-seed:-1 Threads:4 Precision:3
[ ][CORE][28/04/25-22:52:14][INFO] Running Real Analysis
[ ][CORE][28/04/25-22:52:14][INFO] Running Statistical Analysis


100%|██████████| 1000/1000 [03:46<00:00,  4.41it/s]

[ ][CORE][28/04/25-22:56:01][INFO] Building Pvalues result
[ ][CORE][28/04/25-22:56:01][INFO] Building results


Saved deconvoluted to ../output/cpdb/cosmx_cpdb/statistical_analysis_deconvoluted_04_28_2025_225601.txt
Saved deconvoluted_percents to ../output/cpdb/cosmx_cpdb/statistical_analysis_deconvoluted_percents_04_28_2025_225601.txt
Saved means to ../output/cpdb/cosmx_cpdb/statistical_analysis_means_04_28_2025_225601.txt
Saved pvalues to ../output/cpdb/cosmx_cpdb/statistical_analysis_pvalues_04_28_2025_225601.txt
Saved significant_means to ../output/cpdb/cosmx_cpdb/statistical_analysis_significant_means_04_28_2025_225601.txt
cellphone END
2025-04-28 22:56:01.656887


# Visium

In [19]:
###ma ensembl
adata = sc.read_h5ad("../PRIMUS/data/33_data/transfer/Sofie_bc_prace/visium_hd_annotated.h5ad")
adata

AnnData object with n_obs × n_vars = 273789 × 16797
    obs: 'in_tissue', 'array_row', 'array_col', 'location_id', 'region', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'barcode', 'label'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'label_colors', 'log1p', 'spatialdata_attrs'
    obsm: 'spatial'
    layers: 'lognorm'

In [22]:
adata.var

,gene_ids,feature_types,genome,mt,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,n_cells
Xkr4,ENSMUSG00000051951,Gene Expression,mm10,False,5089,0.013351,0.013262,98.706876,5254.0,8.566936,4456
Rp1,ENSMUSG00000025900,Gene Expression,mm10,False,14,0.000036,0.000036,99.996443,14.0,2.708050,12
Sox17,ENSMUSG00000025902,Gene Expression,mm10,False,745,0.001972,0.001970,99.810694,776.0,6.655440,627
Lypla1,ENSMUSG00000025903,Gene Expression,mm10,False,2108,0.005425,0.005410,99.464353,2135.0,7.666690,1844
Tcea1,ENSMUSG00000033813,Gene Expression,mm10,False,12070,0.031943,0.031444,96.932991,12571.0,9.439227,10403
...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd4,ENSMUSG00000064363,Gene Expression,mm10,False,130537,0.439360,0.364198,66.830308,172907.0,12.060514,102133
mt-Nd5,ENSMUSG00000064367,Gene Expression,mm10,False,137032,0.474904,0.388593,65.179917,186895.0,12.138308,108835
mt-Nd6,ENSMUSG00000064368,Gene Expression,mm10,False,22495,0.060016,0.058284,94.283979,23619.0,10.069849,17290
mt-Cytb,ENSMUSG00000064370,Gene Expression,mm10,False,221308,0.964616,0.675297,43.765230,379618.0,12.846924,171615


In [23]:
print("preparing START")
print(datetime.datetime.now())
adata.X = adata.layers['lognorm']
humanized_adata = translate(adata,stay_sparse=True)
humanized_adata.var

humanized_adata.layers['lognorm'] = humanized_adata.X.copy()
sc.pp.normalize_total(humanized_adata, target_sum=1e4)
sc.pp.log1p(humanized_adata)
humanized_adata.layers['lognorm'] = humanized_adata.X.copy()

meta = pd.DataFrame({
    'Cell': humanized_adata.obs_names,
    'cell_type': humanized_adata.obs['label']
})
meta.to_csv("../output/cpdb/visium_meta.txt", sep='\t', index=False)

#something like this will be important for visiumHD because the csv is gonna be HUGE
#not tested
adata_to_use = humanized_adata.copy()
adata_to_use.X = adata_to_use.layers['lognorm']  # or use .X if raw is not available
adata_to_use.var_names_make_unique()

# Create the DataFrame
counts = pd.DataFrame(
    adata_to_use.X.T.toarray() if hasattr(adata_to_use.X, "toarray") else adata_to_use.X.T,
    index=adata_to_use.var_names,
    columns=adata_to_use.obs_names
)

# Save as HDF5
counts.to_hdf("../output/cpdb/visium_counts.h5", key="counts", mode="w")

print("preparing END")
print(datetime.datetime.now())

preparing START
2025-04-30 10:31:41.294555


100%|██████████| 16797/16797 [06:39<00:00, 41.99it/s]   


Found direct orthologs for 14789 genes.
Found multiple orthologs for 303 genes.
Found no orthologs for 846 genes.
Found no index in biomart for 859 genes.


  0%|          | 0/136 [00:00<?, ?it/s]/usr/lib/python3/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


ValueError: The file '../output/cpdb/visium_counts.h5' is already opened.  Please close it before reopening in write mode.

In [18]:
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method
from cellphonedb.src.core.methods import cpdb_analysis_method


print("cellphone START")
print(datetime.datetime.now())

cpdb_results = cpdb_statistical_analysis_method.call(
    cpdb_file_path="../data/cellphonedb.zip",  # use default database (or provide path to custom .zip)
    meta_file_path="../output/cpdb/visium_meta.txt",
    counts_file_path="../output/cpdb/visium_counts.h5",
    counts_data="ensembl",  # or 'ensembl' or gene_name
    output_path="../output/cpdb/visium_cpdb")


print("cellphone END")
print(datetime.datetime.now())

cellphone START
2025-04-30 09:18:32.144457
Reading user files...


AttributeError: Attribute 'block0_items_variety' does not exist in node: '/counts'